In [19]:
import pandas as pd

RATING_COLUMNS = {
    "Helpful": "Helpful",
    "Answerable_relevant": "Answerable/relevant",
    "Harmful_wrong": "Harmful/wrong",
    "Need_context": "Need context"
}

df = pd.read_excel("2023_cosc_304_chatbot.xlsx")

for key, col in RATING_COLUMNS.items():
    df[col] = pd.to_numeric(df[col], errors='coerce')
    df[col] = df[col].apply(lambda x: x if x in [0, 1] else 0)

total_questions = len(df)
valid_questions = df[df[RATING_COLUMNS["Answerable_relevant"]] == 1]
num_valid_questions = len(valid_questions)
num_helpful_answers = valid_questions[RATING_COLUMNS["Helpful"]].sum()
pct_helpful = (num_helpful_answers / num_valid_questions) * 100 if num_valid_questions > 0 else 0

num_need_context = df[RATING_COLUMNS["Need_context"]].sum()
pct_need_context = (num_need_context / total_questions) * 100

num_harmful = df[RATING_COLUMNS["Harmful_wrong"]].sum()
pct_harmful = (num_harmful / total_questions) * 100

num_invalid = len(df[df[RATING_COLUMNS["Answerable_relevant"]] == 0])
pct_invalid = (num_invalid / total_questions) * 100

metrics = {
    "Percentage of Helpful Answers (Excluding Invalid Questions)": round(pct_helpful, 2),
    "Percentage of Questions Needing Context": round(pct_need_context, 2),
    "Percentage of Harmful Questions": round(pct_harmful, 2),
    "Percentage of Invalid Questions": round(pct_invalid, 2)
}

for metric, value in metrics.items():
    print(f"{metric}: {value}%")

Percentage of Helpful Answers (Excluding Invalid Questions): 53.23%
Percentage of Questions Needing Context: 72.15%
Percentage of Harmful Questions: 10.13%
Percentage of Invalid Questions: 21.52%


In [17]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from IPython.display import display, HTML

# Define the path to your annotated Excel file
ANNOTATED_EXCEL_FILE = "./2024_11_28_304_annotated.xlsx"

# Load the Excel file into a pandas DataFrame
try:
    df = pd.read_excel(ANNOTATED_EXCEL_FILE)
    print(f"Successfully loaded '{ANNOTATED_EXCEL_FILE}'.")
except FileNotFoundError:
    print(f"Error: File '{ANNOTATED_EXCEL_FILE}' does not exist.")
except Exception as e:
    print(f"Error loading Excel file: {e}")

# Display the first few rows to inspect the data
df.head()

Successfully loaded './2024_11_28_304_annotated.xlsx'.


,id,questionText,responseText,Helpful,Answerable/relevant,Harmful/wrong,Need context,isPreviousQuestion,id.1,timestamp,...,hideInsights,id.3,userId,courseId,role,expires,timestamp.1,userScore,suggested,interaction
0,31.0,how to join or connect course with prairelearning,To join or connect your course with PrairieLea...,1,1,0,1,0,32.0,2024-09-05 09:55:38.832,...,NaN,204.0,177.0,20.0,student,0.0,2024-09-05 09:55:41.36,0.0,0.0,32.0
1,32.0,can i get the link or code,To obtain the enrollment code or link for your...,0,1,0,1,0,33.0,2024-09-05 09:56:41.284,...,NaN,204.0,177.0,20.0,student,0.0,2024-09-05 09:56:43.9,0.0,0.0,33.0
2,35.0,How do I pass the exam,"To pass the exam, you need to ensure that you ...",1,1,0,1,1,36.0,2024-09-05 14:16:47.009,...,NaN,208.0,189.0,20.0,student,0.0,2024-09-05 14:16:47.025,0.0,0.0,36.0
3,36.0,Foreign keys are primary keys in one relation ...,"No, that's not correct. Foreign keys are attri...",1,1,0,0,0,37.0,2024-09-05 16:06:51.327,...,NaN,211.0,192.0,20.0,student,0.0,2024-09-05 16:06:53.093,0.0,0.0,37.0
4,37.0,"Define relation integrity, what is a good data...",Relational integrity refers to the accuracy an...,1,1,0,1,0,38.0,2024-09-05 16:08:42.756,...,NaN,211.0,192.0,20.0,student,0.0,2024-09-05 16:08:45.675,0.0,0.0,38.0


In [22]:
import pandas as pd

ANNOTATED_EXCEL_FILE = "./2024_11_28_304_annotated.xlsx"

df = pd.read_excel(ANNOTATED_EXCEL_FILE)

df['suggested'] = pd.to_numeric(df['suggested'], errors='coerce').fillna(0).astype(int)
df['isPreviousQuestion'] = pd.to_numeric(df['isPreviousQuestion'], errors='coerce').fillna(0).astype(int)

total_queries = len(df)
percent_prev = (df['isPreviousQuestion'].sum() / total_queries) * 100
percent_suggested = (df['suggested'].sum() / total_queries) * 100
percent_both = ((df['isPreviousQuestion'] == 1) & (df['suggested'] == 1)).sum() / total_queries * 100

print(f"Percentage of Previous Questions: {percent_prev:.2f}%")
print(f"Percentage of Suggested: {percent_suggested:.2f}%")
print(f"Percentage of Both: {percent_both:.2f}%")

Percentage of Previous Questions: 20.84%
Percentage of Suggested: 0.00%
Percentage of Both: 0.00%


In [18]:
# Define the rating columns
RATING_COLUMNS = {
    "Helpful": "Helpful",
    "Answerable_relevant": "Answerable/relevant",
    "Harmful_wrong": "Harmful/wrong",
    "Need_context": "Need context"
}

import numpy as np

# Function to clean and validate rating columns
def clean_rating_columns(df, rating_cols):
    """
    Converts rating columns to numeric and ensures they contain only 0 or 1.
    Non-convertible values are set to 0.
    """
    for key, col in rating_cols.items():
        # Convert to numeric, coercing errors to NaN
        df[col] = pd.to_numeric(df[col], errors='coerce')
        
        # Identify non-binary values
        invalid_entries = df[~df[col].isin([0, 1]) & df[col].notna()]
        if not invalid_entries.empty:
            print(f"Warning: Column '{col}' contains non-binary values at rows: {invalid_entries.index.tolist()}")
        
        # Replace NaN and invalid entries with 0
        df[col] = df[col].apply(lambda x: x if x in [0, 1] else 0)
    
    print("Rating columns have been cleaned and validated.")
    return df

# Apply the cleaning function
df = clean_rating_columns(df, RATING_COLUMNS)

# Verify that all rating columns contain only 0 or 1
df[RATING_COLUMNS.values()].nunique()
def calculate_metrics(df, rating_cols):
    """
    Calculates the specified metrics:
    1. Percentage of Helpful Answers (Excluding Invalid Questions)
    2. Percentage of All Questions Needing Context
    3. Percentage of All Harmful Questions
    4. Percentage of Invalid Questions
    """
    total_questions = len(df)
    print(f"Total Questions: {total_questions}")
    
    # Metric 1: Percentage of Helpful Answers (Excluding Invalid Questions)
    valid_questions = df[df[rating_cols["Answerable_relevant"]] == 1]
    num_valid_questions = len(valid_questions)
    num_helpful_answers = valid_questions[rating_cols["Helpful"]].sum()
    pct_helpful = (num_helpful_answers / num_valid_questions) * 100 if num_valid_questions > 0 else 0
    
    # Metric 2: Percentage of All Questions Needing Context
    num_need_context = df[rating_cols["Need_context"]].sum()
    pct_need_context = (num_need_context / total_questions) * 100
    
    # Metric 3: Percentage of All Harmful Questions
    num_harmful = df[rating_cols["Harmful_wrong"]].sum()
    pct_harmful = (num_harmful / total_questions) * 100
    
    # Metric 4: Percentage of Invalid Questions
    num_invalid = len(df[df[rating_cols["Answerable_relevant"]] == 0])
    pct_invalid = (num_invalid / total_questions) * 100
    
    metrics = {
        "Percentage of Helpful Answers (Excluding Invalid Questions)": round(pct_helpful, 2),
        "Percentage of Questions Needing Context": round(pct_need_context, 2),
        "Percentage of Harmful Questions": round(pct_harmful, 2),
        "Percentage of Invalid Questions": round(pct_invalid, 2)
    }
    
    return metrics

# Calculate metrics
metrics = calculate_metrics(df, RATING_COLUMNS)
metrics
def display_metrics(metrics):
    """
    Displays metrics in a readable format.
    """
    from IPython.display import display, HTML
    html = "<h2>Annotation Analysis Metrics</h2><ul>"
    for metric, value in metrics.items():
        html += f"<li><strong>{metric}:</strong> {value}%</li>"
    html += "</ul>"
    display(HTML(html))

# Display the metrics
display_metrics(metrics)

Rating columns have been cleaned and validated.
Total Questions: 451


In [10]:
import os
import pandas as pd
from openai import OpenAI
import time
from tqdm import tqdm
from pydantic import BaseModel, ValidationError
from typing import List


# Initialize the OpenAI client
client = OpenAI()

# Define the path to your Excel files
INPUT_EXCEL_FILE = "./2024_11_28_304.xlsx"
OUTPUT_EXCEL_FILE = "./2024_11_28_304_annotated.xlsx"

# Rating columns
RATING_COLUMNS = ["Helpful", "Answerable/relevant", "Harmful/wrong", "Need context"]

# OpenAI API parameters
MODEL = "gpt-4o-2024-08-06"  # Updated to use the latest gpt-4o model supporting Structured Outputs
MAX_RETRIES = 5               # Max number of retries for API calls
SLEEP_INTERVAL = 2            # Seconds to wait between retries


# -------------------------- Define Pydantic Model -------------------------
class QA_Ratings(BaseModel):
    Helpful: int
    Answerable_relevant: int
    Harmful_wrong: int
    Need_context: int

    class Config:
        schema_extra = {
            "example": {
                "Helpful": 1,
                "Answerable_relevant": 1,
                "Harmful_wrong": 0,
                "Need_context": 0
            }
        }


# ------------------------------ Functions ----------------------------------
def load_excel(file_path: str) -> pd.DataFrame:
    """
    Load the Excel file into a pandas DataFrame.
    """
    if not os.path.exists(file_path):
        print(f"Error: File '{file_path}' does not exist.")
        return None

    try:
        df = pd.read_excel(file_path)
        print(f"Successfully loaded '{file_path}'.")
        return df
    except Exception as e:
        print(f"Error loading Excel file: {e}")
        return None


def save_excel(df: pd.DataFrame, file_path: str):
    """
    Save the DataFrame back to an Excel file.
    """
    try:
        df.to_excel(file_path, index=False)
        print(f"Successfully saved annotated data to '{file_path}'.")
    except Exception as e:
        print(f"Error saving Excel file: {e}")


def construct_messages(question: str, response: str) -> List[dict]:
    """
    Construct the messages payload for the OpenAI API with Structured Outputs.
    """
    system_message = "You are an assistant that categorizes QA pairs into specific ratings based on the following criteria."
    user_message = (
        "Analyze the following Question and its Response, and provide binary (1 for Yes, 0 for No) ratings for the following categories:\n\n"
        "1. Helpful: Does the answer help resolve the question/task?\n"
        "2. Answerable/relevant: Is the question valid and relevant?\n"
        "3. Harmful/wrong: Does the answer misdirect students or is it harmful?\n"
        "4. Need context: Does the answer require specific context for a language model to respond accurately?\n\n"
        "Provide the ratings in a JSON format adhering to the following schema:\n"
    )

    user_prompt = f"Question:\n{question}\n\nResponse:\n{response}\n\nRatings:"

    return [
        {"role": "system", "content": system_message},
        {"role": "user", "content": user_message + "\n" + user_prompt}
    ]


def get_structured_ratings(question: str, response: str) -> QA_Ratings:
    """
    Send the prompt to OpenAI API and retrieve the structured ratings.
    Implements retries with exponential backoff for robustness.
    """
    messages = construct_messages(question, response)
    for attempt in range(1, MAX_RETRIES + 1):
        try:
            response_obj = client.beta.chat.completions.parse(
                model=MODEL,
                messages=messages,
                response_format=QA_Ratings
            )
            # Extract the parsed response
            parsed = response_obj.choices[0].message.parsed
            print(parsed)
            return parsed
        except ValidationError as ve:
            print(f"Attempt {attempt}: Validation error - {ve}. Retrying after {SLEEP_INTERVAL} seconds...")
        except Exception as e:
            print(f"Attempt {attempt}: Unexpected error - {e}. Retrying after {SLEEP_INTERVAL} seconds...")
        time.sleep(SLEEP_INTERVAL * attempt)  # Exponential backoff
    # After max retries, return default values or handle as needed
    print("Max retries exceeded. Assigning default ratings (0) to avoid blocking.")
    return QA_Ratings(
        Helpful=0,
        Answerable_relevant=0,
        Harmful_wrong=0,
        Need_context=0
    )


def annotate_dataframe(df: pd.DataFrame) -> pd.DataFrame:
    """
    Annotate the DataFrame with ratings by sending each row to the OpenAI API.
    """
    # Initialize columns if they don't exist
    for col in RATING_COLUMNS:
        if col not in df.columns:
            df[col] = None

    # Use tqdm for a progress bar
    for index, row in tqdm(df.iterrows(), total=df.shape[0], desc="Annotating rows"):
        question = row.get("questionText", "")
        response = row.get("responseText", "")

        # Check for NaN and convert to strings
        question = str(question).strip() if pd.notna(question) else ""
        response = str(response).strip() if pd.notna(response) else ""

        # Skip rows where both question and response are empty
        if not question and not response:
            continue

        # Check if the row is already annotated
        if any(pd.isna(row[col]) for col in RATING_COLUMNS):
            print(f"\nProcessing Row {index + 1}:")
            print(f"Q: {question if question else 'N/A'}")
            print(f"A: {response if response else 'N/A'}")

            ratings = get_structured_ratings(question, response)
            df.at[index, "Helpful"] = ratings.Helpful
            df.at[index, "Answerable/relevant"] = ratings.Answerable_relevant
            df.at[index, "Harmful/wrong"] = ratings.Harmful_wrong
            df.at[index, "Need context"] = ratings.Need_context

            print(f"Ratings: {ratings.dict()}")

            # Periodically save progress
            if (index + 1) % 50 == 0:
                print(f"Processed {index + 1} rows. Saving progress...")
                save_excel(df, OUTPUT_EXCEL_FILE)
        else:
            # Skip rows that are already annotated
            continue
    return df


# ---------------------------------- Main -----------------------------------

def main():
    print("Starting the annotation process with Structured Outputs...")
    df = load_excel(INPUT_EXCEL_FILE)
    if df is None:
        print("Terminating due to failed Excel load.")
        return

    df = annotate_dataframe(df)
    save_excel(df, OUTPUT_EXCEL_FILE)


if __name__ == "__main__":
    main()

/Users/kevin/projects/chatbot_analysis/venv/lib/python3.12/site-packages/pydantic/_internal/_config.py:345: UserWarning: Valid config keys have changed in V2:
* 'schema_extra' has been renamed to 'json_schema_extra'
  warnings.warn(message, UserWarning)


Starting the annotation process with Structured Outputs...
Successfully loaded './2024_11_28_304.xlsx'.


Annotating rows:   0%|          | 0/467 [00:00<?, ?it/s]


Processing Row 16:
Q: how to join or connect course with prairelearning
A: To join or connect your course with PrairieLearn, you typically need to follow these steps:

1. **Access PrairieLearn**: Log in to the PrairieLearn platform using your university credentials.
2. **Find Your Course**: Look for the course that you are enrolled in within the PrairieLearn interface.
3. **Enrollment Code**: If required, obtain the enrollment code from your course instructor or syllabus.
4. **Join the Course**: Enter the enrollment code in PrairieLearn to officially join the course.

For specific instructions, refer to any course announcements or contact your instructor.


/var/folders/x2/3kx_8_012rjdwr8pb9vgxmcw0000gn/T/ipykernel_67548/661139870.py:161: PydanticDeprecatedSince20: The `dict` method is deprecated; use `model_dump` instead. Deprecated in Pydantic V2.0 to be removed in V3.0. See Pydantic V2 Migration Guide at https://errors.pydantic.dev/2.10/migration/
  print(f"Ratings: {ratings.dict()}")
Annotating rows:   3%|▎         | 16/467 [00:00<00:22, 20.11it/s]

...

Helpful=1 Answerable_relevant=1 Harmful_wrong=0 Need_context=0
Ratings: {'Helpful': 1, 'Answerable_relevant': 1, 'Harmful_wrong': 0, 'Need_context': 0}

Processing Row 466:
Q: Do lab bonus marks carry over to the overall grade?
A: Lab bonus marks carry over to the overall grade. The labs are worth 20%. If you get 22 out of 20, the extra 2% goes to your overall grade in the course. This does not apply to quiz questions (iClicker, Canvas, Live PrairieLearn). The maximum you can get on that component is 10%.


Annotating rows: 100%|█████████▉| 466/467 [05:31<00:00,  1.41it/s]

Helpful=1 Answerable_relevant=1 Harmful_wrong=0 Need_context=0
Ratings: {'Helpful': 1, 'Answerable_relevant': 1, 'Harmful_wrong': 0, 'Need_context': 0}

Processing Row 467:
Q: Are there any make-up exams or ways to improve my mark on the midterm exam?
A: There are no make-up midterm exams. However, my course policy is
that if you do better on the final exam then that replaces your mark on one or both midterms. 

For example, if you got 70% on the midterm and 80% on the final, your midterm mark is changed to 80%. Even if your mark on the midterm was lower than you wanted, you still can improve that mark by doing well on the final.

Also, any bonus marks on labs, including if they result in over 100%, go to your overall grade. For example, if you get 22/20 for the assignment component, that extra 2% goes directly to your overall grade.

I am available to answer any questions you have and help you if you feel that your midterm did not reflect your abilities.


Annotating rows: 100%|██████████| 467/467 [05:31<00:00,  1.41it/s]

Helpful=1 Answerable_relevant=1 Harmful_wrong=0 Need_context=0
Ratings: {'Helpful': 1, 'Answerable_relevant': 1, 'Harmful_wrong': 0, 'Need_context': 0}
Successfully saved annotated data to './2024_11_28_304_annotated.xlsx'.
